# Kepler visualizations

You can use the `%%mosaic_kepler` magic function to visualise data using [Kepler.gl](https://kepler.gl/).

The mosaic_kepler magic function accepts four parameters:

1) `dataset`: Can be a Spark dataset or a string representing a table/view name

2) `column_name`: The column that needs to be plotted, can be either a geometry column (`WKT`, `WKB` or Mosaic internal format) or a column containing a spatial grid index ID

3) `feature_type`: The type of data to be plotted. Valid values are `geometry` and `h3`

4) `limit`: The maximum number of objects to plot. The default limit is `1000`

Usage:
```
%%mosaic_kepler
dataset column_name feature_type [limit]
```

This magic function is only available in python. It can be used from notebooks with other default languages by storing the intermediate result in a temporary view, and then adding a python cell that uses the `mosaic_kepler` with the temporary view created from another language.

## Examples

In [0]:
%pip install databricks-mosaic --quiet

In [0]:
from pyspark.sql.functions import *
import mosaic as mos
mos.enable_mosaic(spark, dbutils)

### Download example shapes

In [0]:
import requests

req = requests.get('https://data.cityofnewyork.us/api/geospatial/d3c5-ddgc?method=export&format=GeoJSON')
with open('/dbfs/tmp/nyc_taxi_zones.geojson', 'wb') as f:
  f.write(req.content)

In [0]:
neighbourhoods = (
  spark.read
    .option("multiline", "true")
    .format("json")
    .load("dbfs:/tmp/nyc_taxi_zones.geojson")
    
    # Extract geoJSON values for shapes
    .select("type", explode(col("features")).alias("feature"))
    .select("type", col("feature.properties").alias("properties"), to_json(col("feature.geometry")).alias("geom_json"))
    
    # Mosaic internal representation
    .withColumn("geom_internal", mos.st_geomfromgeojson("geom_json"))
  
    # WKT representation
    .withColumn("geom_wkt", mos.st_aswkt(col("geom_internal")))
  
    # WKB representation
    .withColumn("geom_wkb", mos.st_aswkb(col("geom_internal")))
  
   # Limit to only 1 shape
   .limit(1)
)

neighbourhoods.show()

### Plot geometries from Spark dataset

#### Internal geometry type

In [0]:
%%mosaic_kepler
neighbourhoods "geom_internal" "geometry"

![mosaic kepler map example geometry](../images/kepler-1.png)

#### WKT geometry type

In [0]:
%%mosaic_kepler
neighbourhoods "geom_wkt" "geometry"

![mosaic kepler map example geometry](../images/kepler-1.png)

#### WKB geometry type

In [0]:
%%mosaic_kepler
neighbourhoods "geom_wkb" "geometry"

![mosaic kepler map example geometry](../images/kepler-1.png)

### Plot geometries from table/view

In [0]:
neighbourhoods.createOrReplaceTempView("temp_view_neighbourhoods")

In [0]:
%%mosaic_kepler
"temp_view_neighbourhoods" "geom_wkt" "geometry"

![mosaic kepler map example geometry](../images/kepler-1.png)

### Plot H3 indexes

In [0]:
neighbourhood_chips = (neighbourhoods
                       .limit(1)
                       .select(mos.grid_tessellateexplode("geom_internal", lit(9)))
                       .select("index.*")
                    )

neighbourhood_chips.show()

In [0]:
%%mosaic_kepler
neighbourhood_chips "index_id" "h3"

![mosaic kepler map example H3 indexes](../images/kepler-2.png)

### Plot H3 chips

In [0]:
%%mosaic_kepler
neighbourhood_chips "wkb" "geometry"

![mosaic kepler map example h3 chips](../images/kepler-3.png)